In [2]:
import requests
import pandas as pd
import json
import tqdm

In [6]:
url = 'https://www.bundestag.de/ajax/filterlist/de/parlament/plenum/abstimmung/484422-484422?&offset={offset}&noFilterSet=false&view=resultjson'
params = dict(
)

In [5]:
stop
gesetze = []
for offset in range(0,10_000,10):
    resp = requests.get(url=url.format(offset=offset), params=params)
    gesetze.extend(json.loads(resp.content.decode("utf-8"))['items'])
    print(len(gesetze))

    if len(json.loads(resp.content.decode("utf-8"))['items']) == 0:
        break


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450


KeyboardInterrupt: 

In [4]:
gesetze[0]

{'date': '2024-12-19',
 'leading-title': 'Geschäftsordnung',
 'view-variant': 'na',
 'votes': {'no': 69, 'yes': 600, 'abstain': 0, 'absent': 64},
 'href': '/parlament/plenum/abstimmung/abstimmung?id=939',
 'teaser-title': 'Gesetzentwurf zur Änderung von Artikel 93 und 94 des Grundgesetzes (Bundesverfassungsgericht)'}

In [5]:
df = {'date':[],'title':[],'id':[],'topic':[],'url':[]}

for gesetz in gesetze:
    df['date'].append(gesetz['date'])
    df['title'].append(gesetz['teaser-title'])
    df['topic'].append(gesetz['leading-title'])
    df['url'].append(gesetz['href'])
    df['id'].append(gesetz['href'].split('id=')[-1])

df = pd.DataFrame(df)



In [6]:
df.head()

,date,title,id,topic,url
0,2024-12-19,Gesetzentwurf zur Änderung von Artikel 93 und ...,939,Geschäftsordnung,/parlament/plenum/abstimmung/abstimmung?id=939
1,2024-12-16,Antrag gemäß Artikel 68 des Grundgesetzes (Ver...,938,Parlament,/parlament/plenum/abstimmung/abstimmung?id=938
2,2024-10-18,Änderungsantrag zur „Verbesserung der inneren ...,929,Inneres,/parlament/plenum/abstimmung/abstimmung?id=929
3,2024-10-18,Artikel 5 des Entwurfs „Verbesserung der inner...,930,Inneres,/parlament/plenum/abstimmung/abstimmung?id=930
4,2024-10-18,Gesetzentwurf „Verbesserung der Terrorismusbek...,931,Inneres,/parlament/plenum/abstimmung/abstimmung?id=931


In [7]:
df['drucksache'] = None

In [8]:
def get_drucksachen(url):
    url = 'https://www.bundestag.de/' + url
    resp = requests.get(url=url.format(offset=offset), params=params)

    #print(resp.content.decode("utf-8"))
    gesetz = resp.content.decode("utf-8")


    drucksachen = gesetz.split('(Druck')[-1].split('</article> <div class=')[0]
    drucksachen = drucksachen.split('href="')[1:]
    drucksachen = [d.split('"')[0] for d in drucksachen]
    return drucksachen

In [10]:
for i,row in tqdm.tqdm(df.iterrows()):
    row['drucksache'] = get_drucksachen(row['url'])

0it [00:00, ?it/s]

758it [05:27,  2.31it/s]


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))

In [28]:
for i,row in tqdm.tqdm(df.iterrows()):
    if row['drucksache'] is None:
        row['drucksache'] = get_drucksachen(row['url'])
        


866it [00:27, 31.37it/s]  


In [29]:
count = 0
for i,row in tqdm.tqdm(df.iterrows()):
    if row['drucksache'] is None:
        count += 1

count

866it [00:00, 29051.63it/s]


0

In [30]:
df.to_csv('gesetze.csv')

## Absitmmungsdaten und Abgeordnete:

In [3]:
df = pd.read_csv('gesetze.csv')

In [4]:
website = 'https://www.bundestag.de/apps/na/namensliste.form?id={id}'
website

'https://www.bundestag.de/apps/na/namensliste.form?id={id}'

In [7]:
id = df['id'][0]

print(website.format(id=id))
resp = requests.get(url=website.format(id=id), params=params)
resp = resp.content.decode("utf-8")
resp

https://www.bundestag.de/apps/na/namensliste.form?id=939


'\n<div class="meta-top-content">\n    <div class="col-xs-12 bt-bar-chart">\n        <div class="bt-chart-canvas-wrap">\n            <canvas data-chart-values="600,69,0,64"\n                    data-chart-type="bar"></canvas>\n        </div>\n\n\n        <div class="bt-teaser-text-chart">\n                <h3>\n                        Gesamtergebnis,\n                    733 Mitglieder\n                </h3>\n            <ul class="bt-chart-legend">\n                <li class="bt-legend-ja"><span>600</span> Ja</li>\n                <li class="bt-legend-nein"><span>69</span> Nein</li>\n                <li class="bt-legend-enthalten"><span>0</span> Enthalten</li>\n                <li class="bt-legend-na"><span>64</span> Nicht abg.</li>\n            </ul>\n        </div>\n    </div>\n</div>\nWahlperiode: 20\n<div class="meta-slider-content">\n                    <div class="col-xs-4 col-sm-3 col-md-2 bt-slide">\n                        <div class="bt-slide-content">\n\n\n\n               

In [12]:
def get_votes(id):
    website = 'https://www.bundestag.de/apps/na/namensliste.form?id={id}'
    
    resp = requests.get(url=website.format(id=id), params=params)
    resp = resp.content.decode("utf-8")

    names = [r.split('>')[-1] for r in resp.split('</h3>\n\n                                        <p class="bt-person-fraktion">')[:-1]]
    images = [r.split('"')[0] for r in resp.split('data-img-md-normal="')[1:]]
    fractions = [r.split('<')[0] for r in resp.split('class="bt-person-fraktion">')[1:]]
    votes = [r.split('>')[1].split('<')[0] for r in resp.split('bt-person-abstimmung bt-abstimmung')[1:]]

    return {'names':names,'images':images,'fractions':fractions,'votes':votes}
#
#print(len(names))
#print(len(images))
#print(len(fractions))
#print(len(votes))
#names
#fractions
#votes
#images
#votes = ['nicht abgegeben' if v == '' else v for v in votes]
#print(votes[:10])
#print(names[:10])
#print(fractions[:10])
#print(images[:10])

In [13]:
get_votes(930)

{'names': ['Abdi, Sanae',
  'Abel, Valentin',
  'Abraham, Knut',
  'Adler, Katja',
  'Aeffner, Stephanie',
  'Ahmetović, Adis',
  'Akbulut, Gökay',
  'Al-Dailami, Ali',
  'Al-Halak, Muhanad',
  'Alabali-Radovan, Reem',
  'Albani, Stephan',
  'Alt, Renata',
  'Altenkamp, Norbert Maria',
  'Amthor, Philipp',
  'Amtsberg, Luise',
  'Andres, Dagmar',
  'Annen, Niels',
  'Arlt, Johannes',
  'Aschenberg-Dugnus, Christine',
  'Audretsch, Andreas',
  'Auernhammer, Artur',
  'Aumer, Peter',
  'Außendorf, Maik',
  'Bacherle, Tobias B.',
  'Bachmann, Carolin',
  'Badum, Lisa',
  'Baehrens, Heike',
  'Baerbock, Annalena',
  'Bahr, Ulrike',
  'Baldy, Daniel',
  'Banaszak, Felix',
  'Bär, Dorothee',
  'Bär, Karl',
  'Baradari, Nezahat',
  'Bareiß, Thomas',
  'Bartelt, Christian',
  'Bartol, Sören',
  'Bartsch, Dr. Dietmar',
  'Bartz, Alexander',
  'Bas, Bärbel',
  'Bauer, Nicole',
  'Baum, Dr. Christina',
  'Baumann, Dr. Bernd',
  'Bayram, Canan',
  'Beck, Katharina',
  'Beckamp, Roger',
  'Becker, 

In [14]:
abstimmungen = {}
abgeordente = {}
for i,gesetz in tqdm.tqdm(df.iterrows()):
    votes = get_votes(gesetz['id'])
    abstimmungen[gesetz['id']] = votes

0it [00:00, ?it/s]

866it [07:28,  1.93it/s]


In [43]:
with open('abstimmungen.txt','w') as f:
    json.dump(abstimmungen,f)

In [15]:
len(abstimmungen.keys())

866

In [16]:
abstimmungen[930].keys()

dict_keys(['names', 'images', 'fractions', 'votes'])

In [17]:
abgeordente = {}
for gesetz_id in abstimmungen.keys():
    for i in range(len(abstimmungen[gesetz_id]['names'])):
        name = abstimmungen[gesetz_id]['names'][i]
        image = abstimmungen[gesetz_id]['images'][i]
        fractions = abstimmungen[gesetz_id]['fractions'][i]

        abgeordente[name] = {'image': image,'fractions':fractions,'bio':'lorem ipsum'}

print(len(abgeordente))
abgeordente


1590


{'Abdi, Sanae': {'image': 'https://www.bundestag.de/resource/image/861934/3x4/170/227/5b26c151e0c52140dbe629d01df7397f/Kw/abdi_sanae.jpg',
  'fractions': 'SPD',
  'bio': 'lorem ipsum'},
 'Abel, Valentin': {'image': 'https://www.bundestag.de/resource/image/862872/3x4/170/227/f66f94cd04f3ee6eac01ee1525b105a/Fy/Abel_Valentin.png',
  'fractions': 'FDP',
  'bio': 'lorem ipsum'},
 'Abraham, Knut': {'image': 'https://www.bundestag.de/resource/image/861936/3x4/170/227/78214591bf344436613ca4fd179899f8/Oq/abraham_knut.jpg',
  'fractions': 'CDU/CSU',
  'bio': 'lorem ipsum'},
 'Adler, Katja': {'image': 'https://www.bundestag.de/resource/image/861938/3x4/170/227/35b690e42619c3c2d54fa341b1cb8fef/yz/adler_katja.jpg',
  'fractions': 'FDP',
  'bio': 'lorem ipsum'},
 'Aeffner, Stephanie': {'image': 'https://www.bundestag.de/resource/image/861940/3x4/170/227/b0a4aa7dd1ff57fc0b22878ac8f7302e/WP/aeffner_stephanie.jpg',
  'fractions': 'B90/Grüne',
  'bio': 'lorem ipsum'},
 'Ahmetović, Adis': {'image': 'http

In [18]:
stop
url_abg = 'http://spacefaultier.de:8080/api/abgeordnete'
id = 1
for name in abgeordente:
    data = {'name' : name ,
            'partei' : abgeordente[name]['fractions'],
            'bio' : abgeordente[name]['bio']}
    
    abgeordente[name]['id'] = id
    id += 1
    print(json.dumps(data))


    #headers = {"Content-Type": "application/json"}
    response = requests.post(url_abg, data=json.dumps(data))
    print(response)

NameError: name 'stop' is not defined

In [26]:
id = 1
for name in abgeordente:
    abgeordente[name]['id'] = id
    id += 1

In [27]:
len(abgeordente)

1590

In [28]:
abgeordente['Abdi, Sanae']

{'image': 'https://www.bundestag.de/resource/image/861934/3x4/170/227/5b26c151e0c52140dbe629d01df7397f/Kw/abdi_sanae.jpg',
 'fractions': 'SPD',
 'bio': 'lorem ipsum',
 'id': 1}

In [30]:
abgeordente['Lafontaine, Oskar']

{'image': 'http://webarchiv.bundestag.de/archive/2013/1212/bundestag/abgeordnete17/mdbjpg/l/lafontaine_oskar.jpg',
 'fractions': 'Die Linke',
 'bio': 'lorem ipsum',
 'id': 1590}

In [31]:
df['description'] = 'lorem ipsum '*10
df['details'] = 'lorem ipsum '*10
df.head()

,Unnamed: 0,date,title,id,topic,url,drucksache,description,details
0,0,2024-12-19,Gesetzentwurf zur Änderung von Artikel 93 und ...,939,Geschäftsordnung,/parlament/plenum/abstimmung/abstimmung?id=939,['https://dserver.bundestag.de/btd/20/129/2012...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...
1,1,2024-12-16,Antrag gemäß Artikel 68 des Grundgesetzes (Ver...,938,Parlament,/parlament/plenum/abstimmung/abstimmung?id=938,['https://dserver.bundestag.de/btd/20/141/2014...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...
2,2,2024-10-18,Änderungsantrag zur „Verbesserung der inneren ...,929,Inneres,/parlament/plenum/abstimmung/abstimmung?id=929,['https://dserver.bundestag.de/btd/20/128/2012...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...
3,3,2024-10-18,Artikel 5 des Entwurfs „Verbesserung der inner...,930,Inneres,/parlament/plenum/abstimmung/abstimmung?id=930,['https://dserver.bundestag.de/btd/20/128/2012...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...
4,4,2024-10-18,Gesetzentwurf „Verbesserung der Terrorismusbek...,931,Inneres,/parlament/plenum/abstimmung/abstimmung?id=931,['https://dserver.bundestag.de/btd/20/128/2012...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...


In [112]:
stop
df['id'] = 0
url_ges = 'http://spacefaultier.de:8080/api/gesetze'
id = 1
for i,row in df.iterrows():


    topic = row['topic']
    if not isinstance(topic, str):
        topic = ''

    data = {'titel' : row['title'] ,
            'date' : row['date'],
            'topic' : topic,
            'description':  row['description'],
            'details': row['details']}
    
    row['id'] = id
    print(topic)
    print(json.dumps(data))


    #headers = {"Content-Type": "application/json"}
    response = requests.post(url_ges, data=json.dumps(data))
    print(response)
    
    id += 1
    print(id)

Geschäftsordnung
{"titel": "Gesetzentwurf zur \u00c4nderung von Artikel 93 und 94 des Grundgesetzes (Bundesverfassungsgericht)", "date": "2024-12-19", "topic": "Gesch\u00e4ftsordnung", "description": "lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum ", "details": "lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum "}
<Response [200]>
2
Parlament
{"titel": "Antrag gem\u00e4\u00df Artikel 68 des Grundgesetzes (Vertrauensfrage)", "date": "2024-12-16", "topic": "Parlament", "description": "lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum ", "details": "lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum lorem ipsum "}
<Response [200]>
3
Inneres
{"titel": "\u00c4nderungsantrag zur \u201eVerbesserung der inneren Sicherheit\u201c"

In [38]:
[i for i in range(1,len(df['gesetz_id']) +1)]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [39]:
df['gesetz_id'] = [i for i in range(1,len(df['gesetz_id'])+1)]

In [40]:
df['id_old'] = df['id']
df.head()

,Unnamed: 0,date,title,id,topic,url,drucksache,description,details,gesetz_id,id_old
0,0,2024-12-19,Gesetzentwurf zur Änderung von Artikel 93 und ...,939,Geschäftsordnung,/parlament/plenum/abstimmung/abstimmung?id=939,['https://dserver.bundestag.de/btd/20/129/2012...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,1,939
1,1,2024-12-16,Antrag gemäß Artikel 68 des Grundgesetzes (Ver...,938,Parlament,/parlament/plenum/abstimmung/abstimmung?id=938,['https://dserver.bundestag.de/btd/20/141/2014...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,2,938
2,2,2024-10-18,Änderungsantrag zur „Verbesserung der inneren ...,929,Inneres,/parlament/plenum/abstimmung/abstimmung?id=929,['https://dserver.bundestag.de/btd/20/128/2012...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,3,929
3,3,2024-10-18,Artikel 5 des Entwurfs „Verbesserung der inner...,930,Inneres,/parlament/plenum/abstimmung/abstimmung?id=930,['https://dserver.bundestag.de/btd/20/128/2012...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,4,930
4,4,2024-10-18,Gesetzentwurf „Verbesserung der Terrorismusbek...,931,Inneres,/parlament/plenum/abstimmung/abstimmung?id=931,['https://dserver.bundestag.de/btd/20/128/2012...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,lorem ipsum lorem ipsum lorem ipsum lorem ipsu...,5,931


In [93]:
#df.to_csv('gesetze_mit_id.csv')

In [95]:
#with open('abgeordnete_mit_id.txt','w') as f:
#    json.dump(abgeordente,f)

In [46]:
url_abst = 'http://spacefaultier.de:8080/api/abstimmungen'
count = 1
for i,row in df.iterrows():
    gesetz_id = row['gesetz_id']
    id_old = int(row['id_old'])
    for j in range(len(abstimmungen[id_old]['names'])):

        
        if gesetz_id > 80:
            break

        if count <= 58173:
            count += 1
        else:
            print(j)
            name = abstimmungen[id_old]['names'][j]
            abgeordente_id = abgeordente[name]['id']
            vote = abstimmungen[id_old]['votes'][j]

            
            data = {'gesetzId' : gesetz_id,
                    'abgeordneteId' : abgeordente_id,
                    'vote' : vote}
            
            print(json.dumps(data))

            response = requests.post(url_abst, data=json.dumps(data))
            print(response)
        


118
{"gesetzId": 80, "abgeordneteId": 120, "vote": ""}
<Response [200]>
119
{"gesetzId": 80, "abgeordneteId": 121, "vote": " Ja"}
<Response [200]>
120
{"gesetzId": 80, "abgeordneteId": 122, "vote": " Ja"}
<Response [200]>
121
{"gesetzId": 80, "abgeordneteId": 123, "vote": " Ja"}
<Response [200]>
122
{"gesetzId": 80, "abgeordneteId": 124, "vote": " Ja"}
<Response [200]>
123
{"gesetzId": 80, "abgeordneteId": 125, "vote": " Ja"}
<Response [200]>
124
{"gesetzId": 80, "abgeordneteId": 126, "vote": " Ja"}
<Response [200]>
125
{"gesetzId": 80, "abgeordneteId": 127, "vote": " Ja"}
<Response [200]>
126
{"gesetzId": 80, "abgeordneteId": 128, "vote": " Ja"}
<Response [200]>
127
{"gesetzId": 80, "abgeordneteId": 129, "vote": " Ja"}
<Response [200]>
128
{"gesetzId": 80, "abgeordneteId": 130, "vote": " Ja"}
<Response [200]>
129
{"gesetzId": 80, "abgeordneteId": 131, "vote": " Ja"}
<Response [200]>
130
{"gesetzId": 80, "abgeordneteId": 132, "vote": " Nein"}
<Response [200]>
131
{"gesetzId": 80, "abge

In [ ]:
url_abst = 'http://spacefaultier.de:8080/api/abstimmungen/multi'
count = 1
for i,row in df.iterrows():
    gesetz_id = row['gesetz_id']
    id_old = int(row['id_old'])

    if gesetz_id > 80:
        all_data = []
        for j in range(len(abstimmungen[id_old]['names'])):
        
            print(j)
            name = abstimmungen[id_old]['names'][j]
            abgeordente_id = abgeordente[name]['id']
            vote = abstimmungen[id_old]['votes'][j]

            
            data = {'gesetzId' : gesetz_id,
                    'abgeordneteId' : abgeordente_id,
                    'vote' : vote}
            
            all_data.append(data)
                
        #print(json.dumps(all_data))

        response = requests.post(url_abst, data=json.dumps(all_data))
        print(response)